# Kaggle - Help to Protect the Great Barrier Reef

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
save_path = '/content/drive/MyDrive/Protect_Reef_Model'

In [2]:
trained_model_path = '/content/drive/MyDrive/Protect_Reef_Model/best.pt'

In [ ]:
# Ensembling Model Check
# model1_path = '/content/drive/MyDrive/Protect_Reef_Process/yolov5s6/weights/best.pt'
# model2_path = '/content/drive/MyDrive/Protect_Reef_Process/yolov5m6/weights/best.pt'

**Make sure to run COTS_download.ipynb, COTS_previsualization.ipynb and one of the model trainin notebook ahead or download dataset to your drive, change the relative paths, change the label to YOLO format and train the model.**

## Environment Preparation

### Install Libraries

In [3]:
! pip install wandb -qqq
! pip install -qU bbox-utility # check https://github.com/awsaf49/bbox for source code

     |████████████████████████████████| 1.8 MB 9.0 MB/s 
     |████████████████████████████████| 144 kB 67.9 MB/s 
     |████████████████████████████████| 181 kB 76.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 58 kB 4.1 MB/s 
     |████████████████████████████████| 3.3 MB 13.9 MB/s 
     |████████████████████████████████| 34.5 MB 1.3 MB/s 


### Import Libraries

In [4]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob

import shutil
import sys

from joblib import Parallel, delayed

from IPython.display import display


In [5]:
# Use Weights & Biases (W&B) MLOps platform for tracking the experiments
# Log in to your W&B account

import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## YOLOV5 Setting

In [6]:
os.chdir('/content')
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-137-gd876caa torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.2/166.8 GB disk)


## Model Validation

In [7]:
!python val.py --weights {trained_model_path} --data /content/drive/MyDrive/Protect_Reef_Model/gbr.yaml --img 3008 --iou 0.6 --half

val: data=/content/drive/MyDrive/Protect_Reef_Model/gbr.yaml, weights=['/content/drive/MyDrive/Protect_Reef_Model/best.pt'], batch_size=32, imgsz=3000, conf_thres=0.001, iou_thres=0.5, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-137-gd876caa torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 280 layers, 12308200 parameters, 0 gradients, 16.2 GFLOPs
val: Scanning '/content/drive/MyDrive/Protect_Reef_Model/val.cache' images and labels... 2099 found, 0 missing, 0 empty, 0 corrupt: 100% 2099/2099 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:   0% 0/66 [00:00<?, ?it/s]tcmalloc: large alloc 2038431744 bytes == 0x55cbe09c0000 @  0x7f66d8d33b6b 0x7f66d8d53379 0x7f65d360fcde 0x7f65d3611452 0x7f6625638cf3 0x7f66256c0158 0x7f6625d

In [ ]:
# Ensembling Validation
# !python val.py --weights {model1_path} {model2_path} --data /content/drive/MyDrive/Protect_Reef_Model/gbr.yaml --img 3000 --iou 0.6 --half

## Output File

In [8]:
OUTPUT_DIR = 'runs/val/exp'
!ls {OUTPUT_DIR}

confusion_matrix.png  R_curve.png	     val_batch1_pred.jpg
F1_curve.png	      val_batch0_labels.jpg  val_batch2_labels.jpg
P_curve.png	      val_batch0_pred.jpg    val_batch2_pred.jpg
PR_curve.png	      val_batch1_labels.jpg


In [10]:
shutil.copytree(OUTPUT_DIR, save_path + '/result')

'/content/drive/MyDrive/Protect_Reef_Model/result'

### Ground True V.S. Predict

In [ ]:
fig, ax = plt.subplots(3, 2, figsize = (2*9,3*5), constrained_layout = True)
for row in range(3):
    ax[row][0].imshow(plt.imread(f'{OUTPUT_DIR}/val_batch{row}_labels.jpg'))
    ax[row][0].set_xticks([])
    ax[row][0].set_yticks([])
    ax[row][0].set_title(f'{OUTPUT_DIR}/val_batch{row}_labels.jpg', fontsize = 12)
    
    ax[row][1].imshow(plt.imread(f'{OUTPUT_DIR}/val_batch{row}_pred.jpg'))
    ax[row][1].set_xticks([])
    ax[row][1].set_yticks([])
    ax[row][1].set_title(f'{OUTPUT_DIR}/val_batch{row}_pred.jpg', fontsize = 12)
plt.show()

## Result

### Confusion Matrix

In [ ]:
plt.figure(figsize=(12,10))
plt.axis('off')
plt.imshow(plt.imread(f'{OUTPUT_DIR}/confusion_matrix.png'));

### Metrics

In [ ]:
for metric in ['F1', 'PR', 'P', 'R']:
    print(f'Metric: {metric}')
    plt.figure(figsize=(12,10))
    plt.axis('off')
    plt.imshow(plt.imread(f'{OUTPUT_DIR}/{metric}_curve.png'));
    plt.show()